# Libraries

In [1]:
!pip list

Package                            Version
---------------------------------- -------------------
alabaster                          0.7.12
altair                             4.2.0
altair-data-server                 0.4.1
altair-saver                       0.5.0
altair-viewer                      0.4.0
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.10.0
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.0
astropy                            4.2.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              19.3.0
autopep8                           1.5.6
Babel                              2.9.1
backcall                           0.1.0
backports.functools-lru-cache      1.

You should consider upgrading via the 'c:\users\jshernandezm\anaconda3\python.exe -m pip install --upgrade pip' command.



bkcharts                           0.2
black                              19.10b0
bleach                             3.1.5
bokeh                              2.3.2
boto                               2.49.0
boto3                              1.17.94
botocore                           1.20.94
Bottleneck                         1.3.2
brotlipy                           0.7.0
certifi                            2021.10.8
cffi                               1.14.5
chardet                            4.0.0
chart-studio                       1.1.0
click                              8.0.1
cloudpickle                        1.6.0
clyent                             1.2.2
colorama                           0.4.3
comtypes                           1.1.10
conda                              4.11.0
conda-build                        3.21.4
conda-content-trust                0+unknown
conda-pack                         0.6.0
conda-package-handling             1.7.3
conda-repo-cli                     1.0.

In [2]:
import multiprocessing as mp

In [ ]:
mp.

In [1]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import pyodbc
import pandas.io.formats.excel
from multiprocessing import Pool
from USA_DWH import main

# Settings
pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

# Paramaters

weeks = []
a = datetime(2021, 1, 3)
for i in range(0, 3):
    b = a + timedelta(days=84)
    weeks.append((a.strftime('%Y%m%d'), b.strftime('%Y%m%d')))
    a = b + timedelta(days=7)

weeks = []
a = datetime(2017, 1, 1)
for i in range(0, 124):
    b = a + timedelta(days=7)
    weeks.append((a.strftime('%Y%m%d'), b.strftime('%Y%m%d')))
    a = b + timedelta(days=7)

In [2]:
country = 'Estados Unidos de América'

#weeks = [('20210103', '20210228'), ('20210307', '20210425'), ('20210502', '20210627'), ('20210704', '20210829'), ('20210905', '20210926')]

## SQL Server

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=no;')

In [4]:
# Connection to DWH
## Server:SFEDWH01|DB:Gnm_DWH
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

## Server:SFEDWH01|DB:Gnm_MasterOp
conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

# Import data from DWH

In [5]:
# Query time data
qtiempo = '''SELECT TmpID, TmpFecha, TmpSemanaAnioGenomma, TmpMesGenomma, TmpAnioSemanaGenomma
FROM Gnm_DWH.dbo.Dim_Tiempo WHERE TmpAnioSemanaGenomma IN (2018, 2019, 2020, 2021, 2022)'''
# Import time data
dtiempo = pd.read_sql(qtiempo, conn1)

# Query clients data
qclientes= '''SELECT DISTINCT SUC.SucID, SUC.SucCodCliente, CLIE.GrpID, CLIE.GrpNombre, CLIE.CadID, CLIE.CadNombre, SUC.SucNombre
 FROM (SELECT DISTINCT PaisNombre, GrpID, GrpNombre, CadID, CadNombre FROM Gnm_MasterOp.dbo.vw_EstructuraClientesSegPTVTotal
	WHERE PaisNombre = '{0}') AS CLIE
 LEFT JOIN (SELECT DISTINCT SucID, SucNombre, SucCodCliente,CadId FROM Gnm_MasterOp.dbo.vw_EstructuraSucursalesTotal ) AS SUC
	ON CLIE.CadID = SUC.CadID
	ORDER BY SucId;'''

# Query skus data
qproductos = '''SELECT DISTINCT MrcNombre, ProID, ProNombre, ProPstCodBarras, ProPstNombre, ProPstID
                FROM Gnm_MasterOp.dbo.vw_EstructuraProductosInternacional
                WHERE TipoComNombre = '{0}' '''

In [6]:
qsemfalta = 'SELECT TmpID FROM Gnm_DWH.dbo.Dim_Tiempo  WHERE TmpID BETWEEN 20211003 AND 20211205 AND TmpDiaSemana = 7'
#qsemfalta = 'SELECT TmpID FROM Gnm_DWH.dbo.Dim_Tiempo  WHERE TmpID = 20211128 AND TmpDiaSemana = 7'
semfalta = pd.read_sql(qsemfalta, conn1)

In [7]:
%%time
# Import clients data
dclientes = pd.read_sql(qclientes.format(country), conn2)
# Clean data
dclientes.dropna(axis=0, inplace=True)
dclientes.reset_index(drop=True, inplace=True)

# Import products
dproductos = pd.read_sql(qproductos.format(country), conn2)

Wall time: 5.86 s


weeks = list(zip(semfalta['TmpID'].tolist(), semfalta['TmpID'].tolist()))

# Merges

In [8]:
params = []
#weeks = [(20211003, 20211003), (20211010, 20211010), (20211017, 20211017), (20211024, 20211024), (20211031, 20211031)]
weeks = [(20220206, 20220213), (20220220, 20220227)]
for winit, wend in weeks:
    first_day = winit
    last_day = wend
    params.append([first_day, last_day, dtiempo, dclientes, dproductos])

In [9]:
%%time
pool = Pool(len(params))
final = pool.starmap(main, params)
pool.close()
pool.join()
del pool

Wall time: 2min 27s


In [10]:
data_final = pd.DataFrame()
for table in final:
    data_final = pd.concat([data_final, table])

In [11]:
data_final['Semana GL'] = data_final['Semana GL'].map(int)

In [12]:
data_final.pivot_table(index='GrpNombre', 
                       columns='Semana GL', 
                       values=['Unidades Ventas'],
                       margins=True,
                       aggfunc='sum')

Unidades Ventas                                 
Semana GL                         6       7       8       9      All
GrpNombre                                                           
AMAZON US                     16039   16131   14385   14800    61355
MIDWAY IMPORTING             119306  105735  100514   71268   396823
VERMONT IP BEVERAGE           52863   46709   37986   45395   182953
Wal-Mart USA                  76251   70980   65745   69694   282670
Walgreens USA                 60063   60675   57684   55200   233622
All                          324522  300230  276314  256357  1157423

In [13]:
data_final.pivot_table(index='GrpNombre', 
                       columns='Semana GL', 
                       values=['Monto Ventas'],
                       margins=True,
                       aggfunc='sum')

Monto Ventas                                         \
Semana GL                      6            7            8            9   
GrpNombre                                                                 
AMAZON US             113,719.75   114,805.43   100,920.96   106,524.32   
MIDWAY IMPORTING      471,564.00   378,249.02   387,641.13   282,073.73   
VERMONT IP BEVERAGE    74,298.30    65,674.85    53,493.95    63,815.70   
Wal-Mart USA          408,280.14   384,709.27   373,484.28   402,247.99   
Walgreens USA         381,457.22   381,124.03   361,228.89   348,957.58   
All                 1,449,319.41 1,324,562.60 1,276,769.21 1,203,619.32   

                                  
Semana GL                    All  
GrpNombre                         
AMAZON US             435,970.46  
MIDWAY IMPORTING    1,519,527.88  
VERMONT IP BEVERAGE   257,282.80  
Wal-Mart USA        1,568,721.68  
Walgreens USA       1,472,767.72  
All                 5,254,270.54

In [14]:
data_final[data_final['GrpNombre']!='MIDWAY IMPORTING'].pivot_table(index='GrpNombre', 
                                                                    columns='Semana GL', 
                                                                    values=['Monto Ventas'],
                                                                    margins=True,
                                                                    aggfunc='sum')

Monto Ventas                                              
Semana GL                      6          7          8          9          All
GrpNombre                                                                     
AMAZON US             113,719.75 114,805.43 100,920.96 106,524.32   435,970.46
VERMONT IP BEVERAGE    74,298.30  65,674.85  53,493.95  63,815.70   257,282.80
Wal-Mart USA          408,280.14 384,709.27 373,484.28 402,247.99 1,568,721.68
Walgreens USA         381,457.22 381,124.03 361,228.89 348,957.58 1,472,767.72
All                   977,755.41 946,313.58 889,128.08 921,545.59 3,734,742.66

# Export

%%time
path = '../../../02Paises/EUA/Consultas/'
filename = 'Consulta {0}_{1}-{2}.xlsx'.format(country, datetime.today().strftime('%Y%m%d'), datetime.today().strftime('%H%M'))
data_final.to_excel(path + filename, index=False, sheet_name=country)

# Sankey

In [15]:
from pdb import set_trace
import plotly.graph_objects as go
import pandas as pd
import random
pd.options.display.float_format = '{:,.2f}'.format

In [16]:
def create_params(df, cols, value):
    labels = df.melt(value_vars=cols).drop_duplicates()['value'].tolist()
    labels_nodes = dict(zip(labels, range(len(labels))))
    source, target, values = [], [], []
    link_color = dict(zip(range(len(labels)), [random_color('rgba') for x in range(len(labels))]))
    for sour, targ in zip(cols, cols[1:]):
        aux = df.pivot_table(index=[sour, targ], values=value, aggfunc='sum').reset_index()
        source += aux[sour].map(labels_nodes).tolist()
        target += aux[targ].map(labels_nodes).tolist()
        values += aux[value].tolist()
    df_trace = pd.DataFrame([source, target, values]).T.rename({0:'source', 1:'target', 2:'value'}, axis=1)
    #df_trace['color'] = df_trace['source'].apply(lambda x: random_color('color'))
    df_trace['link color'] = df_trace['source'].map(link_color)
    return df_trace, list(labels_nodes.keys())

In [17]:
def random_color(method):
    if method == 'rgba':
        return 'rgba' + str(tuple(random.choices(range(256), k=3) + [round(random.uniform(0.2, 0.85), 4)]))
    else:
        return "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])

In [18]:
def sankey_diagram(data_trace, labels, year, weeks):
    fig = go.Figure(data=[go.Sankey(orientation = "h",
                                    valueformat = ",.2f",
                                    node = dict(pad = 15, 
                                                thickness = 20, 
                                                line = dict(color = "black", width = 0.5),
                                                label = labels,
                                                #color = data_trace['color']),
                                                color = 'black'),
                                    link = dict(source = data_trace['source'],
                                                target = data_trace['target'],
                                                value = data_trace['value'],
                                                color = data_trace['link color'])
                                   )],
                    layout={'showlegend': True})
    weeks_text = ', '.join([str(w) for w in weeks])
    fig.update_layout(title_text="Genomma USA - Sell Out Week {1} - {0}".format(year, weeks_text), font_size=12)
    fig.show()

In [19]:
def create_params_perc(df, cols, value):
    labels = df.melt(value_vars=cols).drop_duplicates()['value'].tolist()
    labels_nodes = dict(zip(labels, range(len(labels))))
    source, target, values = [], [], []
    link_color = dict(zip(range(len(labels)), [random_color('rgba') for x in range(len(labels))]))
    for sour, targ in zip(cols, cols[1:]):
        aux = df.pivot_table(index=[sour, targ], values=value, aggfunc='sum').reset_index()
        source += aux[sour].map(labels_nodes).tolist()
        target += aux[targ].map(labels_nodes).tolist()
        total_values = aux[value].sum()
        values += [x/total_values for x in aux[value].tolist()]
    df_trace = pd.DataFrame([source, target, values]).T.rename({0:'source', 1:'target', 2:'value'}, axis=1)
    df_trace['link color'] = df_trace['source'].map(link_color)
    return df_trace, list(labels_nodes.keys())

In [20]:
data_final.columns

Index(['Anio GL', 'Semana GL', 'Mes GL', 'Fecha', 'GrpID', 'GrpNombre',
       'CadID', 'CadNombre', 'MrcNombre', 'ProID', 'ProNombre', 'ProPstID',
       'ProPstCodBarras', 'ProPstNombre', 'Unidades Ventas',
       'Unidades Inventario', 'Monto Ventas', 'Monto Inventario'],
      dtype='object')

In [21]:
data_final['País'] = 'USA'

In [22]:
year = 2022
#weeks = range(36, 39)
weeks = [5]
cols_nodes = ['País', 'MrcNombre']

#data = data_final[(data_final['Año_GL'] == year)&(data_final['Semana_GL'].isin(weeks))&(data_final['MrcNombre']=='TIO NACHO')].copy()
data = data_final[(data_final['Anio GL'] == year)&(data_final['Semana GL'].isin(weeks))].copy()
data.reset_index(drop=True, inplace=True)

trace, label = create_params_perc(df=data, cols=cols_nodes, value='Monto Ventas')
sankey_diagram(trace, label, year, weeks)

KeyError: 'Monto Ventas'

In [ ]:
year = 2022
#weeks = range(36, 39)
weeks = [5]
cols_nodes = ['País', 'MrcNombre']

#data = data_final[(data_final['Año_GL'] == year)&(data_final['Semana_GL'].isin(weeks))&(data_final['MrcNombre']=='TIO NACHO')].copy()
data = data_final[(data_final['Anio GL'] == year)&(data_final['Semana GL'].isin(weeks))&(data_final['GrpNombre'].isin(['Target', 'Walgreens USA', 'Wal-Mart USA', 'AMAZON US', 'VERMONT IP BEVERAGE']))].copy()
data.reset_index(drop=True, inplace=True)

trace, label = create_params_perc(df=data, cols=cols_nodes, value='Monto Ventas')
sankey_diagram(trace, label, year, weeks)